# Imports

In [1]:
import numpy as np
import pandas as pd

# Acquire
- bring in all the data
    - calendar.csv
    - sales_train_validation.csv
    - sell_prices.csv  

In [10]:
calendar = pd.read_csv("calendar.csv")
sales = pd.read_csv("sales_train_validation.csv")
sell_prices = pd.read_csv("sell_prices.csv")

# Prepare

## Plan for Data Preparation:
1. Summarize the data
    - How many records do we have?
    - What data types do we have?
    - Are there columns that should become multiple columns?
    - Is a column a string w/ delimeters?
    - A column containing JSON or lists might need expansion
    - Are there many columns that should become two columns (.melt)
    - Do we need to do any joins w/ other tables?
    - What exactly is an "observation"?
    - What does each row represent?
2. Handle nulls
    - When to remove?
    - The row (individual observation)
    - The column (the entire feature)
    - When to replace nulls?
    - What to replace nulls with?
4. Handle Anomalies/Outliers
    - Our first anomaly filter is the IQR Rule


### Summarizing calendar dataframe

In [11]:
calendar.head(4)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0


In [12]:
calendar.describe()

,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI
count,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000
mean,11347.086338,3.997461,6.325546,2013.288471,0.330117,0.330117,0.330117
std,155.277043,2.001141,3.416864,1.580198,0.470374,0.470374,0.470374
min,11101.000000,1.000000,1.000000,2011.000000,0.000000,0.000000,0.000000
25%,11219.000000,2.000000,3.000000,2012.000000,0.000000,0.000000,0.000000
50%,11337.000000,4.000000,6.000000,2013.000000,0.000000,0.000000,0.000000
75%,11502.000000,6.000000,9.000000,2015.000000,1.000000,1.000000,1.000000
max,11621.000000,7.000000,12.000000,2016.000000,1.000000,1.000000,1.000000


In [13]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
date            1969 non-null object
wm_yr_wk        1969 non-null int64
weekday         1969 non-null object
wday            1969 non-null int64
month           1969 non-null int64
year            1969 non-null int64
d               1969 non-null object
event_name_1    162 non-null object
event_type_1    162 non-null object
event_name_2    5 non-null object
event_type_2    5 non-null object
snap_CA         1969 non-null int64
snap_TX         1969 non-null int64
snap_WI         1969 non-null int64
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


In [25]:
calendar.isnull().sum()

date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
d                  0
event_name_1    1807
event_type_1    1807
event_name_2    1964
event_type_2    1964
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64

In [15]:
calendar.shape

(1969, 14)

#### Thoughts so far?
   - Take a closer look at:
        - event_name_1
        - event_type_1
        - event_name_2
        - event_type_2
    - I am thinking these are holidays

In [32]:
calendar.event_name_1.value_counts()

StPatricksDay          6
LentWeek2              6
MemorialDay            6
Mother's day           6
ValentinesDay          6
NBAFinalsStart         6
NBAFinalsEnd           6
LentStart              6
Purim End              6
SuperBowl              6
Ramadan starts         6
PresidentsDay          6
Pesach End             6
Eid al-Fitr            5
OrthodoxChristmas      5
LaborDay               5
Thanksgiving           5
Chanukah End           5
OrthodoxEaster         5
ColumbusDay            5
MartinLutherKingDay    5
Christmas              5
Halloween              5
Easter                 5
Cinco De Mayo          5
IndependenceDay        5
EidAlAdha              5
NewYear                5
VeteransDay            5
Father's day           4
Name: event_name_1, dtype: int64

In [35]:
calendar.event_type_1.value_counts()

Religious    55
National     52
Cultural     37
Sporting     18
Name: event_type_1, dtype: int64

In [36]:
calendar.event_name_2.value_counts()

Father's day      2
Cinco De Mayo     1
Easter            1
OrthodoxEaster    1
Name: event_name_2, dtype: int64

In [37]:
calendar.event_type_2.value_counts()

Cultural     4
Religious    1
Name: event_type_2, dtype: int64

### Summarizing sales dataframe

In [17]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [18]:
sales.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.070220,1.041292,0.780026,0.833454,0.627944,0.958052,0.918662,1.244080,1.073663,0.838701,...,1.370581,1.586159,1.693670,1.248245,1.232207,1.159167,1.149000,1.328862,1.605838,1.633158
std,5.126689,5.365468,3.667454,4.415141,3.379344,4.785947,5.059495,6.617729,5.917204,4.206199,...,3.740017,4.097191,4.359809,3.276925,3.125471,2.876026,2.950364,3.358012,4.089422,3.812248
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
max,360.000000,436.000000,207.000000,323.000000,296.000000,314.000000,316.000000,370.000000,385.000000,353.000000,...,129.000000,160.000000,204.000000,98.000000,100.000000,88.000000,77.000000,141.000000,171.000000,130.000000


In [19]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [29]:
# Do we have any nulls?
(sales.isnull().sum() >= 1).sum()

0

In [31]:
sales.shape

(30490, 1919)

In [49]:
sales.store_id.value_counts()

TX_3    3049
TX_1    3049
CA_4    3049
CA_2    3049
CA_1    3049
TX_2    3049
WI_2    3049
CA_3    3049
WI_3    3049
WI_1    3049
Name: store_id, dtype: int64

### Summarizing sell_prices dataframe

In [38]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [44]:
for c in sell_prices.columns:
    print(sell_prices[c].value_counts())

TX_2    701214
TX_1    699796
CA_1    698412
WI_3    696094
CA_3    693990
TX_3    691112
CA_4    679025
WI_2    678171
WI_1    665912
CA_2    637395
Name: store_id, dtype: int64
FOODS_3_136        2820
FOODS_1_088        2820
HOUSEHOLD_1_472    2820
HOUSEHOLD_1_474    2820
FOODS_3_793        2820
                   ... 
HOUSEHOLD_1_308     642
HOUSEHOLD_1_159     623
HOUSEHOLD_1_242     600
FOODS_3_296         592
FOODS_2_379         530
Name: item_id, Length: 3049, dtype: int64
11618    30490
11614    30490
11619    30490
11605    30490
11604    30490
         ...  
11105    13033
11104    12799
11103    12519
11102    11975
11101    10932
Name: wm_yr_wk, Length: 282, dtype: int64
2.98     253976
1.98     248204
2.50     193621
3.98     179757
3.97     172302
          ...  
30.32         1
11.23         1
16.18         1
0.13          1
0.81          1
Name: sell_price, Length: 1048, dtype: int64


In [45]:
sell_prices.describe()

,wm_yr_wk,sell_price
count,6.841121e+06,6.841121e+06
mean,1.138294e+04,4.410952e+00
std,1.486100e+02,3.408814e+00
min,1.110100e+04,1.000000e-02
25%,1.124700e+04,2.180000e+00
50%,1.141100e+04,3.470000e+00
75%,1.151700e+04,5.840000e+00
max,1.162100e+04,1.073200e+02


In [46]:
sell_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
store_id      object
item_id       object
wm_yr_wk      int64
sell_price    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [47]:
sell_prices.isnull().sum()

store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64

In [48]:
sell_prices.shape

(6841121, 4)

## Create Dataframes for each Store

In [51]:
sales.store_id.value_counts()

TX_3    3049
TX_1    3049
CA_4    3049
CA_2    3049
CA_1    3049
TX_2    3049
WI_2    3049
CA_3    3049
WI_3    3049
WI_1    3049
Name: store_id, dtype: int64

In [55]:
#Texas stores
tx_1 = sales[sales.store_id == "TX_1"]
tx_2 = sales[sales.store_id == "TX_2"]
tx_3 = sales[sales.store_id == "TX_3"]

#California Stores
ca_1 = sales[sales.store_id == "CA_1"]
ca_2 = sales[sales.store_id == "CA_2"]
ca_3 = sales[sales.store_id == "CA_3"]
ca_4 = sales[sales.store_id == "CA_4"]

#Wisconsin Stores
wi_1 = sales[sales.store_id == "WI_1"]
wi_2 = sales[sales.store_id == "WI_2"]
wi_3 = sales[sales.store_id == "WI_3"]

In [60]:
tx_1

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
12196,HOBBIES_1_001_TX_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,0,0,1,0,1,0,0,2
12197,HOBBIES_1_002_TX_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12198,HOBBIES_1_003_TX_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12199,HOBBIES_1_004_TX_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX,1,2,1,0,...,0,3,2,0,1,0,0,0,2,2
12200,HOBBIES_1_005_TX_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,1,2,2,0,0,2,3,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15240,FOODS_3_823_TX_1_validation,FOODS_3_823,FOODS_3,FOODS,TX_1,TX,2,2,2,2,...,0,1,0,0,0,0,0,0,1,0
15241,FOODS_3_824_TX_1_validation,FOODS_3_824,FOODS_3,FOODS,TX_1,TX,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
15242,FOODS_3_825_TX_1_validation,FOODS_3_825,FOODS_3,FOODS,TX_1,TX,0,0,0,3,...,0,1,1,0,1,1,0,0,0,0
15243,FOODS_3_826_TX_1_validation,FOODS_3_826,FOODS_3,FOODS,TX_1,TX,0,0,0,0,...,0,0,0,0,1,1,1,1,0,0
